In [6]:
import pandas as pd
import numpy as np
from finance_byu.summarize import summary
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
# Read in crsp monthly data

df = pd.read_feather("~/Data/crsp_monthly.ftr")

df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/andrew/Data/crsp_monthly.ftr'

In [145]:
# Cleaning

df['prc'] = abs(df['prc'])

df.sort_values(by=['permno', 'caldt'], inplace=True)

df = df[['permno', 'caldt', 'prc', 'ret']]

df.head()

,permno,caldt,prc,ret
0,10000,1986-01-31,4.37500,NaN
1,10000,1986-02-28,3.25000,-0.257143
2,10000,1986-03-31,4.43750,0.365385
3,10000,1986-04-30,4.00000,-0.098592
4,10000,1986-05-30,3.10938,-0.222656
